In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.globals import set_llm_cache
from langchain.cache import SQLiteCache

set_llm_cache(SQLiteCache("cache.db"))

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

In [2]:
examples = [
    {
        "movie_title": "Titanic",
        "movie_info": """
        movie_title: Titanic,

        info:
        director: James Cameron,
        actors: ["Leonardo DiCaprio", "Kate Winslet", "Billy Zane", "Kathy Bates"],
        budgets: $200 million,
        incomes: $2.2 billion (worldwide gross),
        synopsis: A love story between Jack, a penniless artist, and Rose, an aristocratic young woman, unfolds aboard the RMS Titanic, which meets a tragic end during its maiden voyage in 1912.
        """
    },
    {
        "movie_title": "Interstellar",
        "movie_info": """
        movie_title: Interstellar,

        info:
        director: Christopher Nolan,
        actors: ["Matthew McConaughey", "Anne Hathaway", "Jessica Chastain", "Michael Caine", "Ellen Burstyn"],
        budgets: $165,000,000,
        incomes: $736,723,724,
        synopsis: In a dystopian future where Earth becomes uninhabitable, a former NASA pilot leads a team of astronauts through a wormhole near Saturn to search for a new home for humanity.
        """
    },
    {
        "movie_title": "The Avengers",
        "movie_info": """
        movie_title: The Avengers,

        info:
        director: Joss Whedon,
        actors: [Robert Downey Jr., Chris Evans, Mark Ruffalo, Chris Hemsworth, Scarlett Johansson, Jeremy Renner],
        budgets: $220,000,000,
        incomes: $1,520,538,536,
        synopsis: Earth's mightiest heroes must unite under Nick Fury and S.H.I.E.L.D. to stop Loki and his alien army from enslaving humanity, leading to the formation of the Avengers team.
        """
    }
]

In [7]:
example_prompt = ChatPromptTemplate.from_messages([
    ("human", "Show me a informantion about {movie_title}"),
    ("ai", "{movie_info}")
])

exam_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a expert about movies."),
    exam_prompt,
    ("human", "Show me a informantion about {movie_title}"),
])

chain = final_prompt | chat

chain.invoke({"movie_title": "John Wick"})

AIMessageChunk(content='\n        movie_title: John Wick,\n\n        info:\n        director: Chad Stahelski,\n        actors: ["Keanu Reeves", "Michael Nyqvist", "Alfie Allen", "Willem Dafoe", "Ian McShane"],\n        budgets: $20,000,000,\n        incomes: $88,761,661,\n        synopsis: After the death of his beloved dog, retired hitman John Wick seeks vengeance against the Russian mobsters who wronged him, unleashing his deadly skills in a quest for retribution.\n        ')